In [111]:
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [173]:
def process_text_files(directory):
    all_texts = []
    all_texts_extend = []
    
    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                txt_data = file.read().split()
                all_texts.append(txt_data)
                all_texts_extend.extend(txt_data)
    
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(all_texts_extend)
    
    print(all_texts)
    print("-----------------")
    print(all_texts_extend)
    print("-----------------")

    
    all_sequences=[]
    for i in range(len(all_texts)):
        #print(tokenizer.texts_to_sequences(all_texts[i]))
        all_sequences.append(tokenizer.texts_to_sequences(all_texts[i]))
        
    print("ALL SEQUENCES,",all_sequences)
    print("-----------------")
    
    
    max_word_length = 20
    trunc_type = 'post'
    padding_type = 'post'
    
    # Pad each word sequence to the length of the longest word
    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        if padded_sequence.shape[0] <= 3:
            all_padded_sequences.append(padded_sequence)
        else:
            print("Ignoring sequence with shape greater than 3:", padded_sequence.shape[0])
        
    #new_array=np.array(all_padded_sequences,dtype=object)

    
    return all_padded_sequences, tokenizer.word_index

In [174]:
def add_zero_arrays(arr):
    while len(arr) < 4:
        arr = np.concatenate((arr, np.zeros((1, 20), dtype=np.int32)))
    return arr

In [175]:
directory_path = '/Users/talhaunal/Downloads/OCR_dataset/deneme-veriler'
padded_sequences, word_index = process_text_files(directory_path)

for i in range(len(padded_sequences)):
    print(i,padded_sequences[i])
    
print("-----------------")

print(word_index)

print("-----------------")

#result_list = [add_zero_arrays(arr) for arr in padded_sequences]

#print(result_list)
# Printing the result
#for arr in result_list:
    #print(arr)

[['asd', 'ali'], ['dss', 'cali', 'asd', 'asdasd']]
-----------------
['asd', 'ali', 'dss', 'cali', 'asd', 'asdasd']
-----------------
ALL SEQUENCES, [[[1, 2, 3], [1, 4, 5]], [[3, 2, 2], [6, 1, 4, 5], [1, 2, 3], [1, 2, 3, 1, 2, 3]]]
-----------------
Ignoring sequence with shape greater than 3: 4
0 [[1 2 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 4 5 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
-----------------
{'a': 1, 's': 2, 'd': 3, 'l': 4, 'i': 5, 'c': 6}
-----------------
